# 运动边界非定常流场预测


需要安装 **MindFlow >=0.1.0** 版本

In [ ]:
mindflow_version = "0.1.0"  
!pip uninstall -y mindflow-ascend
!pip install mindflow-ascend==$mindflow_version

## 概述

CFD作为一种通过数值方法来模拟和解析流体运动的重要工具，极大便利了流体力学相关问题的科学研究，在设计、优化和研究领域提供准确的数据和见解并发挥着重要作用。流体力学中具有代表性和研究价值的一类问题是：对具有移动边界的非定常流场系统进行模拟，以分析运动结构在流场中的受力情况，可在工程上优化设计运动结构，为航空航天飞行器以及航海器等外形优化提供方案策略。高精确度计算流体力学技术（CFD）能够准确模拟流场演化和结构受力情况，但是高精度动边界问题需要大量网格，导致硬件消耗和计算时间成本巨大，另外对动态网格的构造也格外耗时。

面对CFD在应用于复杂问题时计算量巨大并且计算精度有待提高等问题，智能流体力学领域给出了行之有效的解决方案，深度学习可以通过深度神经网络可学习流动工况与流场之间的演化关系，快速实现流场高精度预测与重构。为了高效解决动边界流场重构问题，提出了一种混合深度神经网络(HDNN)，以实现非定常动边界流场重构，并基于此实现流场快速预测。

## 问题描述

流场相关尺寸如图所示，其中 $Y = Asin(2πft)$ 代表圆柱体在竖直方向做简谐运动的运动表达式，A为振幅，f为频率；D代表圆柱体直径；矩形边界代表计算域。均匀来流流过运动圆柱体时，在流体与固体相互作用的影响下，会在圆柱体后方形成一系列复杂的流动现象，如边界层分离、交替出现的卡门涡街等，并演化为物理量随时间周期性变化的非均匀流场。


## 模型架构

HDNN的基本框架由卷积神经网络（CNN）、卷积长短期记忆网络（ConvLSTM）和反卷积神经网络（DeCNN）组成。使用CNN结构实现特征提取；ConvLSTM学习低维时空特征并进行预测；最后，DeCNN实现预测流场的重建

+ 输入层：输入历史流场
+ 卷积层：通过多层CNN对输入流场进行降维，提取时空流动特征
+ 记忆层：通过ConvLSTM学习低维空间流场时空特征的演变，预测下一时刻
+ DeCNN输出层：将预测流场的低维特征恢复到高维空间，通过多层DeCNN重构下一时刻的瞬态流场，并输出预测结果

![HDNN.jpg](./images/HDNN.jpg)

## 训练数据集

数据集由非定常二维圆柱绕流的数值仿真流场数据构建的多维矩阵流场快照矩阵构建而成

+ 二维圆柱在均匀来流流场中做一维简谐振动，振动频率f（Hz）分别为1.25、1.43、1.67、2.00，振幅比A/D分别为0.5、0.6、0.7、0.8。两两组合总共16组运动状态
+ 数据集为某一状态（f,A/D）下的非定常流场序列数据
+ 每张流场快照包含3个通道，代表流场的压强分布信息、水平速度信息、竖直速度信息，多维矩阵流场快照矩阵尺寸为：T×C×H×W(C为通道数，H，W分别为snapshot的高和宽）
+ 数据集：[下载位置](https://download.mindspore.cn/mindscience/mindflow/dataset/applications/data_driven/move_boundary_hdnn)

在这里下载数据可以直接通过运行 python download.py

In [5]:
import os
import time
import argparse
import numpy as np

from mindspore import nn, ops, context, save_checkpoint, set_seed, data_sink, jit
from mindflow.utils import load_yaml_config

from src import my_train_dataset, AEnet, save_loss_curve

## 训练环境

+ 训练采用Mindspore框架的静态图模式（GRAPH）
+ 在单卡的Ascend上进行训练


In [6]:
set_seed(0)
np.random.seed(0)

## 训练超参数

从config中获得模型、数据、优化器的超参

In [7]:
# 解析命令行参数
parser = argparse.ArgumentParser()
parser.add_argument("--mode", type=str, default="GRAPH")
parser.add_argument("--save_graphs", type=bool, default=False)
parser.add_argument("--save_graphs_path", type=str, default="./summary")
parser.add_argument("--device_target", type=str, default="Ascend")
parser.add_argument("--device_id", type=int, default=0)
parser.add_argument("--data_list", type=list, default=['0.25', '0.30', '0.35', '0.40'])
parser.add_argument('--batch_size', type=int, default=16, help="batch size")
parser.add_argument("--config_file_path", type=str, default="./config.yaml")

args = parser.parse_args()

# 设置运行上下文
context.set_context(
    mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
    save_graphs=args.save_graphs,
    save_graphs_path=args.save_graphs_path,
    device_target=args.device_target,
    device_id=args.device_id
)
# 判断是否使用 Ascend 设备
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

# 加载配置文件，准备参数
config = load_yaml_config(args.config_file_path)  # 从 YAML 配置文件中读取所有设置
data_params = config["data"]                      # 数据相关参数字典
model_params = config["model"]                    # 模型相关参数字典
optimizer_params = config["optimizer"]            # 优化器相关参数字典

## 训练过程文件保存路径



In [8]:
# 准备保存模型检查点的目录
ckpt_dir = optimizer_params["ckpt_dir"]
if not os.path.exists(ckpt_dir):                  # 若目录不存在则创建
    os.mkdir(ckpt_dir)

## 构建神经网络及优化器

损失函数使用均方误差（Mean Squared Error）损失函数，优化器使用Adam（Adaptive Moment Estimation）优化算法

In [ ]:
# 构建待训练模型
model = AEnet(
    in_channels=model_params["in_channels"],      # 输入通道数
    num_layers=model_params["num_layers"],        # 网络层数
    kernel_size=model_params["kernel_size"],      # 卷积核大小
    num_convlstm_layers=model_params["num_convlstm_layers"]  # ConvLSTM 层数
)
# 定义损失函数：均方误差 MSE
loss_func = nn.MSELoss()
# 定义优化器：Adam
optimizer = nn.Adam(
    params=model.trainable_params(),               # 可训练参数列表
    learning_rate=optimizer_params["lr"]          # 学习率
)

# 如果使用 Ascend 设备，启用动态损失缩放和自动混合精度
if use_ascend:
    from mindspore.amp import DynamicLossScaler, auto_mixed_precision, all_finite
    loss_scaler = DynamicLossScaler(1024, 2, 100)   # 初始化动态损失缩放器
    auto_mixed_precision(model, 'O1')               # 开启 O1 级别混合精度
else:
    loss_scaler = None

## 训练框架

定义前向传播函数forward_fn，将预测值和真值比较得到损失值loss并返回

In [10]:
# 前向计算函数，返回损失
def forward_fn(inputs, velocity, label):
    pred = model(inputs, velocity)                # 前向推理
    loss = loss_func(pred, label)                  # 计算 MSE 损失

    if use_ascend:
        loss = loss_scaler.scale(loss)             # 缩放损失
    return loss

# 计算损失和梯度的函数
grad_fn = ops.value_and_grad(
    forward_fn, None, optimizer.parameters, has_aux=False
)

## 数据集加载

给my_train_dataset传参，得到训练数据集和验证数据集

In [ ]:
# 准备训练和验证数据集
dataset_train, dataset_eval = my_train_dataset(
    data_params["data_dir"],                     # 数据路径
    data_params["time_steps"],                   # 时间步长
    args.data_list,                                # 数据列表（如不同参数设置）
    batch_size=args.batch_size                     # 批大小
)

## 数据下沉及模型训练

定义train_step和eval_step并使用data_sink加速训练，每隔一定训练轮次保存模型文件

In [ ]:
# 定义带 @jit 加速的训练步骤函数
@jit
def train_step(inputs, velocity, label):
    loss, grads = grad_fn(inputs, velocity, label)  # 计算损失和梯度
    if use_ascend:
        loss = loss_scaler.unscale(loss)            # 反缩放损失
        if all_finite(grads):                        # 若梯度有效，则反缩放梯度
            grads = loss_scaler.unscale(grads)
    # 应用更新并返回损失
    loss = ops.depend(loss, optimizer(grads))
    return loss

# 定义带 @jit 加速的验证步骤函数，计算并返回 RMSE 损失
@jit
def eval_step(inputs, velocity, label):
    loss = forward_fn(inputs, velocity, label)     # 计算 MSE 损失
    loss = ops.sqrt(loss)                          # 转为 RMSE
    return loss

# 使用数据下沉机制提升性能
train_sink_process = data_sink(train_step, dataset_train, sink_size=1)
eval_sink_process = data_sink(eval_step, dataset_eval, sink_size=1)
train_data_size = dataset_train.get_dataset_size()  # 训练集批数
eval_data_size = dataset_eval.get_dataset_size()    # 验证集批数

# 用于记录每个 epoch 的平均损失
avg_train_losses = []
avg_valid_losses = []

# 开始循环训练
for epoch in range(1, optimizer_params["epochs"] + 1):
    train_losses = 0.0
    valid_losses = 0.0

    # 记录训练开始时间
    local_time_beg = time.time()
    model.set_train(True)                            # 切换到训练模式

    # 遍历所有训练批次
    for _ in range(train_data_size):
        step_train_loss = ops.squeeze(train_sink_process(), axis=())  # 获取标量损失
        step_train_loss = step_train_loss.asnumpy().item()            # 转为 Python 数值
        train_losses += step_train_loss

    # 计算并记录平均训练损失
    train_loss = train_losses / train_data_size
    avg_train_losses.append(train_loss)
    print(f"epoch: {epoch}, epoch average train loss: {train_loss:.6f}, "
          f"epoch time: {(time.time() - local_time_beg):.2f}s")

    # 每隔一定间隔进行验证
    if epoch % optimizer_params["eval_interval"] == 0:
        eval_time_beg = time.time()
        model.set_train(False)                       # 切换到评估模式
        for _ in range(eval_data_size):
            step_eval_loss = ops.squeeze(eval_sink_process(), axis=())
            step_eval_loss = step_eval_loss.asnumpy().item()
            valid_losses += step_eval_loss

        valid_loss = valid_losses / eval_data_size
        avg_valid_losses.append(valid_loss)
        print(f"epoch: {epoch}, epoch average valid loss: {valid_loss:.6f}, "
              f"epoch time: {(time.time() - eval_time_beg):.2f}s")

    # 每隔一定间隔保存模型
    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(model, f"{ckpt_dir}/net_{epoch}.ckpt")

# 绘制并保存训练与验证损失曲线
save_loss_curve(
    avg_train_losses, 'Epoch', 'avg_train_losses', 'Avg_train_losses Curve', 'Avg_train_losses.png'
)
save_loss_curve(
    avg_valid_losses, 'Epoch', 'avg_valid_losses', 'Avg_valid_losses Curve', 'Avg_valid_losses.png'
)

## 设置训练条件 传参

当运行该文件时，通过参数解析器传入必要参数，开始训练，并打印进程和设备id，以及训练总时间

In [ ]:
if __name__ == "__main__":
    # 记录开始时间并启动训练
    start_time = time.time()
    train()
    print(f"total time: {(time.time() - start_time):.2f}s")

## 评估阶段

In [ ]:
import os
import time
import argparse
import numpy as np
from scipy.io import savemat, loadmat

from mindspore import nn, ops, load_checkpoint, load_param_into_net, set_seed
from mindflow.utils import load_yaml_config

from src import my_test_dataset, AEnet, save_loss_curve

np.random.seed(0)
set_seed(0)

## 读取并设置超参数

In [ ]:
# 解析命令行参数
parser = argparse.ArgumentParser()
parser.add_argument("--mode", type=str, default="GRAPH")           # 运行模式：GRAPH/其他
parser.add_argument("--device_target", type=str, default="Ascend")# 设备目标：Ascend/GPU/CPU
parser.add_argument("--device_id", type=int, default=0)              # 设备 ID
parser.add_argument("--config_file_path", type=str, default="./config.yaml")  # 配置文件路径
parser.add_argument("--infer_mode", type=str, default="one")       # 推理模式：one 或 cycle

args = parser.parse_args()

# 加载配置文件，准备参数
config = load_yaml_config(args.config_file_path)  # 从 YAML 配置文件中读取所有设置
data_params = config["data"]                      # 数据相关参数字典
model_params = config["model"]                    # 模型相关参数字典
prediction_params = config["prediction"]          # 推理相关参数字典
prediction_result_dir = prediction_params["prediction_result_dir"]  # 保存一次性预测结果的目录
pred_continue_dir = prediction_params["pred_continue_dir"]          # 保存循环预测结果的目录

## 构建网络模型并读取模型参数

In [ ]:
 # 构建网络模型
    net = AEnet(
        in_channels=model_params["in_channels"],          # 输入通道数
        num_layers=model_params["num_layers"],            # 网络层数
        kernel_size=model_params["kernel_size"],          # 卷积核大小
        num_convlstm_layers=model_params["num_convlstm_layers"]  # ConvLSTM 层数
    )
    # 加载模型参数
    m_state_dict = load_checkpoint(prediction_params["ckpt_path"])  # 从指定路径加载 checkpoint
    load_param_into_net(net, m_state_dict)  # 将参数加载到网络中

## 准备测试数据集并定义损失函数

In [ ]:
# 准备测试数据集
data_set = my_test_dataset(prediction_params["data_dir"], data_params["time_steps"])  # 创建测试数据迭代器
# 若保存结果的目录不存在，则创建
if not os.path.exists(prediction_result_dir):
    os.mkdir(prediction_result_dir)
if not os.path.exists(pred_continue_dir):
    os.mkdir(pred_continue_dir)

# 定义损失函数：均方误差 MSE
loss_func = nn.MSELoss()

# 保存各步的测试损失
test_losses = []

## 根据参数设置选择预测的模式：单步预测或者循环预测

In [ ]:
# 根据推理模式执行不同流程
# 单步预测模式：predict next one-step flow field
if args.infer_mode == "one":
    for i, (input_1, velocity, label, matrix_01) in enumerate(data_set):
        pred = net(input_1, velocity)                   # 前向推理，获取预测
        pred = ops.mul(pred, matrix_01)                  # 对预测结果应用掩码或变换矩阵
        loss = ops.sqrt(loss_func(pred, label))          # 计算预测值与真实值的均方根误差
        test_losses.append(loss)                         # 记录损失张量
        print(f"test loss: {(loss.asnumpy().item()):.6f}")  # 打印当前损失
        # 保存预测结果与真实值到 mat 文件
        savemat(f"{prediction_result_dir}/prediction_data{i}.mat", {
            'prediction': pred.asnumpy(),
            'real': label.asnumpy(),
        })

# 循环预测模式：predict a complete periodic flow field
elif args.infer_mode == "cycle":
    for i, (inputvar, velocityvar, targetvar, matrix_01) in enumerate(data_set):
        if i == 0:
            inputs = inputvar  # 第一帧输入
        label = targetvar      # 真值
        velocity = velocityvar # 流场速度
        # 前向推理
        pred = net(inputs, velocity)
        pred = ops.mul(pred, matrix_01)                   # 应用矩阵变换
        loss = ops.sqrt(loss_func(pred, label))           # 计算均方根误差
        loss_aver = loss.asnumpy().item()                  # 转为 Python 数值

        # 记录并打印损失
        test_losses.append(loss_aver)
        print(f"test loss: {loss_aver:.6f}")
        # 保存循环预测结果
        savemat(f"{pred_continue_dir}/prediction_data{i}.mat", {
            'prediction': pred.asnumpy(),
            'real': label.asnumpy(),
        })
        # 将当前预测结果拼接为下一步的输入
        pred = ops.operations.ExpandDims()(pred, 1)       # 扩展维度以匹配输入格式
        cat = ops.concat((inputs, pred), axis=1)          # 在时间维度上连接张量
        inputs = cat[:, 1:, :, :, :]                     # 去掉最老的一帧，保留最新序列


## 绘制并保存测试损失曲线

In [ ]:
save_loss_curve(
        test_losses,        # 损失列表
        'Epoch',            # 横轴标签
        'test_losses',      # 纵轴标签
        'Test_losses Curve',# 图表标题
        'Test_losses.png'   # 输出文件名
    )

## 预测流场结果可视化

+ 动边界流场预测通过执行eval.py开始预测，分为两种预测方式：单步流场预测（infer_mode为"one"）和一个振动周期内连续流场预测（infer_mode为"cycle"）；单步流场预测仅预测下一时刻一个时间步长的流场，连续流场预测则持续预测一个完整周期的流场
+ 下图为在振幅比为0.25、0.30、0.35、0.40数据集上训练完备的HDNN模型，在0.45数据集上进行测试得到的结果
+ 具体可以将eval.py处理得到的数据经过draw.py处理，得到dat文件，再使用Tecplot 360软件进行最后的可视化操作

![pred_single_step_puv.png](./images/pred_single_step_puv.png)